# Welcome to the OpenMusenet music generation notebook!
This is a free/open source implementation of Musenet, a midi generation model by OpenAI. If you like this project, make sure to [star the repo on Github](https://github.com/hidude562/OpenMusenet2)
This will either let you generate from scratch or start from a prexisting song.

---

Before running any of the cells on this notebook, goto **Runtime -> Change runtime type -> Hardware accelerator -> GPU** and then save.

## Import some neccesary things and install model

In [1]:
!pip install py_midicsv
!pip install mido
!pip install py_midicsv
!apt install fluidsynth
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2
!pip install transformers

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

Processing triggers for man-db (2.10.2-1) ...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.8 MB/s eta 0:00:00


In [1]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import math
import random

In [2]:
#@title Model Name
model_name = "hidude562/Openmusenet-2.1" #@param ["hidude562/Openmusenet-2.0", "hidude562/Openmusenet-2.1"] {allow-input: true}

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
# Load pre-trained model (weights)
model = GPT2LMHeadModel.from_pretrained(model_name)

## Run this if you want to start from an existing midi **(expand cell)**

In [9]:
# @title Open file
!rm -rf /content/midi
!mkdir -p /content/midi/
%cd /content/midi/

from google.colab import files
uploaded = files.upload()

%cd /content/

import py_midicsv as pm
import os
import re


midi_dir = "/content/midi/"
midi_files = [f for f in os.listdir(midi_dir) if f.endswith('.mid')]
csv_string = []

def get_note_event(start, note, instrument, length, velocity):
  out_instrument = "%"
  match instrument:
    case 2:
      out_instrument = "%"
    case 3:
      out_instrument = "^"
    case 4:
      out_instrument = "&"
    case 5:
      out_instrument = "*"
    case 6:
      out_instrument = "("
    case 7:
      out_instrument = "("
    case _:
      out_instrument = "%"


  dynamic = "$"
  if(velocity > 108):
    dynamic = "$"
  elif velocity > 74:
    dynamic = "#"
  elif velocity > 40:
    dynamic = "@"
  else:
    dynamic = "!"

  return [start, dynamic, length, out_instrument, note]

def parse_split_csv_item(item):
  try:
    item[2] = item[2].replace(' ', '')
    item[0] = int(item[0])
    item[1] = int(item[1])
    item[3] = int(item[3])
    item[4] = int(item[4])
    item[5] = int(item[5])
  except:
    pass;
  return item

# Load the MIDI file and parse it into CSV format
numNotesInCsv = 0
for midi_file in midi_files:
  csv_parsed = []
  try:
    lastNoteStart = 0
    csv_unparsed = pm.midi_to_csv(midi_dir + midi_file)
    notesPlaying = []
    for csv_item in csv_unparsed:
      split_csv_item = csv_item.split(',')
      split_csv_item = parse_split_csv_item(split_csv_item);
      match(split_csv_item[2]):
        case "Note_on_c":
          toAdd = True
          for i in notesPlaying:
            if(i[4] == split_csv_item[4]):
              toAdd = False
          if toAdd:
            notesPlaying.append(split_csv_item)
        case "Note_off_c":
          # Find note that turning off
          i = 0
          while i < len(notesPlaying):
            if(notesPlaying[i][4] == split_csv_item[4]):
              temp = get_note_event(notesPlaying[i][1], split_csv_item[4], split_csv_item[0], split_csv_item[1] - notesPlaying[i][1], notesPlaying[i][5])
              csv_parsed.append(temp)
              #csv_parsed.append(get_note_event(notesPlaying[i][1] - lastNoteStart, split_csv_item[4], 0, split_csv_item[1] - notesPlaying[i][1], notesPlaying[i][5]) + '|')
              #csv_string += get_note_event(notesPlaying[i][1] - lastNoteStart, split_csv_item[4], 0, split_csv_item[1] - notesPlaying[i][1], notesPlaying[i][5]) + '|'
              lastNoteStart = notesPlaying[i][1]
              notesPlaying.pop(i)
              i-=1
            i+=1
  except Exception as e:
    print(e)

  # Change to be delta time instead of abs and sort time
  csv_parsed = sorted(csv_parsed, key=lambda x: x[0])
  for i in range(len(csv_parsed) - 1):
    csv_item_copy = csv_parsed[i].copy()
    if(i != 0):
      csv_item_copy[0] = csv_parsed[i][0] - csv_parsed[i - 1][0]

    # Also concat to csv_parsed
    for j in csv_item_copy:
      csv_string += str(j)
    csv_string += "|"
    numNotesInCsv+=1

#for i in range(0, len(csv_string), 1800):
#  csv_string[i] = "\n"

with open("startMid.txt", "w") as f:
    f.writelines(csv_string)

/content/midi


Saving allca prmpt.mid to allca prmpt.mid
/content


## Run this if you want to start from scratch **(dont run this if you want to start from existing)**

In [6]:
!rm /content/startMid.txt
!touch /content/startMid.txt

## Run this to generate (For generating from both scratch and from existing)
Once you run all these cells, in the Colab file explobarer on the side, there will be a file named "aiOut.mid", this is the AI generated midi file either from scratch or for prompt. Run the next cell if you want to play it back in your browser

In [24]:
#@title Generate
device = torch.device("cuda")
model.cuda()

#generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
#generated = generated.to(device)

def generateMusic(prompt):
  generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
  generated = generated.to(device)

  sample_outputs = model.generate(
                                  generated,
                                  #bos_token_id=random.randint(1,30000),
                                  do_sample=True,
                                  #top_k=130,
                                  max_length = 760,
                                  temperature=0.95,
                                  num_return_sequences=1
                                  )

  #return tokenizer.decode(sample_outputs, skip_special_tokens=True)
  for i, sample_output in enumerate(sample_outputs):
    return tokenizer.decode(sample_output, skip_special_tokens=True)
    print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))
  return ""

musicPrompt = ""
with open("/content/startMid.txt") as f:
  musicPrompt = f.read()

builtInputs = "<|startoftext|>" + musicPrompt
allText = ""
songLen = 5 #@param {type:"integer"}

for i in range(songLen):
  gen = generateMusic(builtInputs[-500:])[len(builtInputs[-500:]):]
  builtInputs += gen
  allText += gen

  finalOutput = builtInputs
  finalOutput = finalOutput[:finalOutput.rfind('|') - 1]
  finalOutput = finalOutput[finalOutput.find('>') + 1:]

print("Output song: ")
print(finalOutput)

f = open("outAI.txt", "w")
f.write(finalOutput + "")
f.close()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

Output song: 
0$65%71|64$65%69|64$65%68|64$65%69|64$84%72|0$130%57|128$121%60|0$121%64|128$65%74|0$84%60|0$84%64|64$65%72|64$65%71|0$84%60|0$84%64|64$65%72|64$84%76|1%60|0$121%64|128$65%75|0$84%60|0$84%64|64$65%76|64$65%79|0$84%60|0$84%64|64$65%78|64$65%76|0$84%60|0$84%64|64$65%78|64$84%80|0$130%52|128$121%56|0$121%62|128$65%81|0$84%56|0$84%62|64$65%80|64$65%78|0$84%56|0$84%62|64$65%80|64$256%81|0$261%57|128$130%60|0$130%64|128$130%60|0$130%64|128$130%60|0$130%64|128$261%57|128$130%60|0$130%64|128$130%60|0$130%64|128$261%48|128$130%57|0$130%60|0$130%66|128$130%60|0$130%63|0$266%47|128$130%57|0$130%60|0$130%66|128$261%40|128$130%56|0$130%59|0$130%64|128$65%64|0$65%59|0$243%44|64$65%68|64$65%71|64$65%74|64$130%76|0$130%40|128$121%52|0$121%68|128$65%73|0$84%52|0$84%57|64$65%76|64$65%81|0$84%52|0$84%57|64$65%80|64$65%78|0$84%52|0$84%57|64$65%80|64$84%81|0$130%48|128$121%51|0$121%57|0$121%60|0$121%66|128$130%51|0$130%57|0$130%60|0$130%66|128$243%47|128$130%51|0$130%57|0$130%60|0$130%66|128$

In [25]:
#@title Midi header info
midi_tracks = []

def reset_midi():
  global midi_tracks
  midi_tracks = [
  '''0, 0, Header, 1, 7, 480
              1, 0, Start_track
              1, 0, Title_t, ""
              1, 0, Time_signature, 4, 2, 24, 8
              1, 0, Tempo, 500000
              1, 0, End_track
              2, 0, Start_track
              2, 0, Control_c, 0, 121, 0
              2, 0, Title_t, ""
              2, 0, Control_c, 0, 10, 64
              2, 0, Control_c, 0, 7, 100
              2, 0, Control_c, 0, 11, 127
              2, 0, Control_c, 0, 101, 0
              2, 0, Control_c, 0, 100, 2
              2, 0, Control_c, 0, 6, 64
              2, 0, Control_c, 0, 101, 0
              2, 0, Control_c, 0, 100, 1
              2, 0, Control_c, 0, 6, 64
              2, 0, Control_c, 0, 38, 0
              2, 0, Control_c, 0, 101, 0
              2, 0, Control_c, 0, 100, 0
              2, 0, Control_c, 0, 6, 12
              2, 0, Pitch_bend_c, 0, 8192
              2, 0, Control_c, 0, 1, 0
              2, 0, Program_c, 0, 0''',
                '''3, 0, Start_track
              3, 0, Control_c, 1, 121, 0
              3, 0, Title_t, ""
              3, 0, Control_c, 1, 10, 64
              3, 0, Control_c, 1, 7, 100
              3, 0, Control_c, 1, 11, 127
              3, 0, Control_c, 1, 101, 0
              3, 0, Control_c, 1, 100, 2
              3, 0, Control_c, 1, 6, 64
              3, 0, Control_c, 1, 101, 0
              3, 0, Control_c, 1, 100, 1
              3, 0, Control_c, 1, 6, 64
              3, 0, Control_c, 1, 38, 0
              3, 0, Control_c, 1, 101, 0
              3, 0, Control_c, 1, 100, 0
              3, 0, Control_c, 1, 6, 12
              3, 0, Pitch_bend_c, 1, 8192
              3, 0, Control_c, 1, 1, 0
              3, 0, Program_c, 1, 88''',
                '''5, 0, Start_track
              5, 0, Control_c, 2, 121, 0
              5, 0, Title_t, ""
              5, 0, Control_c, 2, 10, 64
              5, 0, Control_c, 2, 7, 100
              5, 0, Control_c, 2, 11, 127
              5, 0, Control_c, 2, 101, 0
              5, 0, Control_c, 2, 100, 2
              5, 0, Control_c, 2, 6, 64
              5, 0, Control_c, 2, 101, 0
              5, 0, Control_c, 2, 100, 1
              5, 0, Control_c, 2, 6, 64
              5, 0, Control_c, 2, 38, 0
              5, 0, Control_c, 2, 101, 0
              5, 0, Control_c, 2, 100, 0
              5, 0, Control_c, 2, 6, 12
              5, 0, Pitch_bend_c, 2, 8192
              5, 0, Control_c, 2, 1, 0
              5, 0, Program_c, 2, 29''',
                '''4, 0, Start_track
              4, 0, Control_c, 2, 121, 0
              4, 0, Title_t, ""
              4, 0, Control_c, 2, 10, 64
              4, 0, Control_c, 2, 7, 100
              4, 0, Control_c, 2, 11, 127
              4, 0, Control_c, 2, 101, 0
              4, 0, Control_c, 2, 100, 2
              4, 0, Control_c, 2, 6, 64
              4, 0, Control_c, 2, 101, 0
              4, 0, Control_c, 2, 100, 1
              4, 0, Control_c, 2, 6, 64
              4, 0, Control_c, 2, 38, 0
              4, 0, Control_c, 2, 101, 0
              4, 0, Control_c, 2, 100, 0
              4, 0, Control_c, 2, 6, 12
              4, 0, Pitch_bend_c, 2, 8192
              4, 0, Control_c, 2, 1, 0
              4, 0, Program_c, 2, 0''',
                '''5, 0, Start_track
              5, 0, Control_c, 3, 121, 0
              5, 0, Title_t, ""
              5, 0, Control_c, 3, 10, 64
              5, 0, Control_c, 3, 7, 100
              5, 0, Control_c, 3, 11, 127
              5, 0, Control_c, 3, 101, 0
              5, 0, Control_c, 3, 100, 2
              5, 0, Control_c, 3, 6, 64
              5, 0, Control_c, 3, 101, 0
              5, 0, Control_c, 3, 100, 1
              5, 0, Control_c, 3, 6, 64
              5, 0, Control_c, 3, 38, 0
              5, 0, Control_c, 3, 101, 0
              5, 0, Control_c, 3, 100, 0
              5, 0, Control_c, 3, 6, 12
              5, 0, Pitch_bend_c, 3, 8192
              5, 0, Control_c, 3, 1, 0
              5, 0, Program_c, 3, 1''',
                '''6, 0, Start_track
              6, 0, Control_c, 4, 121, 0
              6, 0, Title_t, ""
              6, 0, Control_c, 6, 10, 64
              6, 0, Control_c, 6, 7, 100
              6, 0, Control_c, 6, 11, 127
              6, 0, Control_c, 6, 101, 0
              6, 0, Control_c, 6, 100, 2
              6, 0, Control_c, 6, 6, 64
              6, 0, Control_c, 6, 101, 0
              6, 0, Control_c, 6, 100, 1
              6, 0, Control_c, 6, 6, 64
              6, 0, Control_c, 6, 38, 0
              6, 0, Control_c, 6, 101, 0
              6, 0, Control_c, 6, 100, 0
              6, 0, Control_c, 6, 6, 12
              6, 0, Pitch_bend_c, 6, 8192
              6, 0, Control_c, 6, 1, 0
              6, 0, Program_c, 6, 0''',
                '''7, 0, Start_track
              7, 0, Control_c, 7, 121, 0
              7, 0, Title_t, ""
              7, 0, Control_c, 7, 10, 64
              7, 0, Control_c, 7, 7, 100
              7, 0, Control_c, 7, 11, 127
              7, 0, Control_c, 7, 101, 0
              7, 0, Control_c, 7, 100, 2
              7, 0, Control_c, 7, 6, 64
              7, 0, Control_c, 7, 101, 0
              7, 0, Control_c, 7, 100, 1
              7, 0, Control_c, 7, 6, 64
              7, 0, Control_c, 7, 38, 0
              7, 0, Control_c, 7, 101, 0
              7, 0, Control_c, 7, 100, 0
              7, 0, Control_c, 7, 6, 12
              7, 0, Pitch_bend_c, 7, 8192
              7, 0, Control_c, 7, 1, 0
              7, 0, Program_c, 7, 3''',
                '''8, 0, Start_track
              2, 0, Control_c, 9, 121, 0
            2, 0, Title_t, ""
            2, 0, Control_c, 9, 10, 64
            2, 0, Control_c, 9, 7, 100
            2, 0, Control_c, 9, 11, 127
            2, 0, Control_c, 9, 101, 0
            2, 0, Control_c, 9, 100, 2
            2, 0, Control_c, 9, 6, 64
            2, 0, Control_c, 9, 101, 0
            2, 0, Control_c, 9, 100, 1
            2, 0, Control_c, 9, 6, 64
            2, 0, Control_c, 9, 38, 0
            2, 0, Control_c, 9, 101, 0
            2, 0, Control_c, 9, 100, 0
            2, 0, Control_c, 9, 6, 12
            2, 0, Pitch_bend_c, 9, 8192
            2, 0, Control_c, 9, 1, 0
            2, 0, Program_c, 9, 0'''
  ]

def add_midi_ends():
  for i in range(len(midi_tracks)):
    midi_tracks[i] += "\n" + str(i + 2) + ", 0, End_track"
def midi_to_string():
  mid_str = ""
  for i in midi_tracks:
    mid_str += i + "\n"
  mid_str += "0, 0, End_of_file"
  return mid_str
reset_midi()

In [26]:
#@title Convert to Midi
reset_midi()

def generate_on_off_events(noteData):
  track_num = 2
  match noteData[3]:
    case "%":
        track_num = 2
    case "^":
        track_num = 3
    case "%":
        track_num = 4
    case "^":
        track_num = 5
    case "&":
        track_num = 6
    case "*":
        track_num = 7
    case "(":
        track_num = 8
    case ")":
        track_num = 9
  #TODO:

  dynamic = 127
  match noteData[3]:
    case "$":
      dynamic = 127
    case "#":
      dynamic = 80
    case "@":
      dynamic = 55
    case "!":
      dynamic = 32

  note_on = str(track_num) + ", " + str(noteData[0]) + ", Note_on_c, 0, " + str(noteData[4]) + ", " + str(dynamic)
  note_off = str(track_num) + ", " + str(noteData[2] + noteData[0]) + ", Note_off_c, 0, " + str(noteData[4]) + ", " + "0"
  return [[noteData[0], note_on, track_num], [noteData[0] + noteData[2], note_off, track_num]]


def test_if_num(num):
  if(num == "-"):
    return True
  try:
    tmp = int(num)
  except:
    return False

  return True

file = open("outAI.txt", "r").read()
fileTokens = file.split("|")

if(len(fileTokens[-1]) < 5):
  fileTokens.pop()
if(len(fileTokens[0]) < 5):
  fileTokens.pop(0)

allParsedVals = []

timeMarker = 0
for i in range(len(fileTokens)):
  valuesArr = []
  valueBuild = ""
  for j in range(len(fileTokens[i])):
    if(test_if_num(fileTokens[i][j])):
      valueBuild += fileTokens[i][j]
    else:
      if(valueBuild != ""):
        valuesArr.append(int(valueBuild))
      valuesArr.append(fileTokens[i][j])
      valueBuild = ""
  if(valueBuild != ""):
    valuesArr.append(int(valueBuild))
    valueBuild = ""
  try:
    oldTime = valuesArr[0]
  except:
    break
  valuesArr[0] += timeMarker
  allParsedVals.append(valuesArr)
  timeMarker += oldTime

noteEventsArray = []
for val in allParsedVals:
  try:
    if(val[4] < 128):
      events = generate_on_off_events(val)

      noteEventsArray.append(events[0])
      noteEventsArray.append(events[1])
  except Exception as e:
    print("AI messed up syntax 💀 (still continuing)", e)
    pass
noteEventsArray = sorted(noteEventsArray, key=lambda x: x[0])
#print(noteEventsArray)

outString = ""
for val in noteEventsArray:
  midi_tracks[val[2] - 2] += "\n" + val[1]
  #midi_tracks[val[2][2] - 2] += "\n" +  val[1][1]
add_midi_ends()
outString = midi_to_string()

with open('mididata.csv', 'w') as file:
    file.write(outString)

import py_midicsv as pm

csv_string = "/content/mididata.csv"

# Parse the CSV output of the previous command back into a MIDI file
midi_object = pm.csv_to_midi(csv_string)

# Save the parsed MIDI file to disk
with open("aiOut.mid", "wb") as output_file:
    print("Saved mid successfully")
    midi_writer = pm.FileWriter(output_file)
    midi_writer.write(midi_object)

AI messed up syntax 💀 (still continuing) list index out of range
Saved mid successfully


## Play midi in Colab (unhide cell) (also sometimes doesn't work)

In [27]:
!fluidsynth -ni font.sf2 aiOut.mid -F output.wav -r 44100
from IPython.display import Audio
Audio('output.wav')

FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file 'output.wav'..


KeyboardInterrupt: ignored